In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import itertools

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
df = pd.read_csv("airline_passengers.csv", index_col="Month", parse_dates=True)
df.index.freq = 'MS'
df.head()

In [ ]:
df.shape

In [ ]:
# Assume the forecast horizon we care about is 12
# Validate over 10 steps
h = 12
steps = 10
Ntest = len(df) - h - steps + 1
print(f"h is {h}, steps is {steps}, Ntest is {Ntest}")

In [ ]:
# configuration hyperparameters to try
trend_type_list = ["add","mul"]
seasonal_type_list = ["add", "mul"]
damped_trend_list = [True, False]
init_method_list = ["estimated","heuristic","legacy-heuristic"]
use_boxcox_list = [True, False, 0]

In [ ]:
def walkforward(trend_type, seasonal_type,damped_trend,init_method,use_boxcox,debug=False):
    errors=[]
    seen_last = False
    steps_completed = 0

    for end_of_train in range(Ntest,len(df)-h+1):
        # we don't have to manually add the data to our dataset
        # just index it at the right points - this is a view not a copy
        # so it doesn't take up any extra space or computation
        train = df.iloc[:end_of_train]
        test = df.iloc[end_of_train : end_of_train + h]

        if test.index[-1] == df.index[-1]:
            seen_last = True
        
        steps_completed += 1

        hw = ExponentialSmoothing(train["Passengers"], initialization_method=init_method, trend=trend_type,damped_trend=damped_trend, seasonal=seasonal_type,seasonal_periods=12,use_boxcox=use_boxcox)
        res_hw = hw.fit()

        # compute error for the forecast horizon
        fcast = res_hw.forecast(h)
        error = mean_squared_error(test["Passengers"],fcast)
        errors.append(error)

        if debug:
            print(f"seen_last : {seen_last}")
            print(f"steps completed : {steps_completed}")
    
    return np.mean(errors)

In [ ]:
walkforward("add","add",False,"legacy-heuristic",0,debug=True)

In [ ]:
# iterate through all possible options
tuple_of_option_lists = (trend_type_list,seasonal_type_list,damped_trend_list,init_method_list,use_boxcox_list)

for x in itertools.product(*tuple_of_option_lists):
    print(x)

In [ ]:
best_score = float("inf")
best_options = None

for x in itertools.product(*tuple_of_option_lists):
    score = walkforward(*x)

    if score < best_score:
        best_score = score
        best_options = x
        print(f"best score so far : {best_score}, with options : {best_options}")


In [ ]:
print(f"best score is {best_score}")
trend_type,seasonal_type,damped_trend,init_method,use_boxcox=best_options
print(f"trend type : {trend_type}")
print(f"seasonal type : {seasonal_type}")
print(f"damped_trend : {damped_trend}")
print(f"initialization_method : {init_method}")
print(f"use boxcox : {use_boxcox}")